In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model
from keras.layers import Lambda
from keras.layers import Input, Dense, Dropout, Multiply, Softmax, Lambda, Concatenate
from keras import backend as K
from sklearn.metrics import roc_auc_score
import random
import warnings


warnings.filterwarnings("ignore")
random.seed(3)
np.random.seed(3)

batch_size = 1024
seed = 3
tf.set_random_seed(seed)

def auc_roc(y_true, y_pred):
    def fallback_auc(y_true, y_pred):
        try:
            return roc_auc_score(y_true, y_pred)
        except:
            return 0.5
    return tf.py_func(fallback_auc, (y_true, y_pred), tf.double)

def to_categorical(y, num_classes=None, dtype='float32'):
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=dtype)
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical

def data_preparation():
    column_names = ['age', 'class_worker', 'det_ind_code', 'det_occ_code', 'education', 'wage_per_hour', 'hs_college',
                    'marital_stat', 'major_ind_code', 'major_occ_code', 'race', 'hisp_origin', 'sex', 'union_member',
                    'unemp_reason', 'full_or_part_emp', 'capital_gains', 'capital_losses', 'stock_dividends',
                    'tax_filer_stat', 'region_prev_res', 'state_prev_res', 'det_hh_fam_stat', 'det_hh_summ',
                    'instance_weight', 'mig_chg_msa', 'mig_chg_reg', 'mig_move_reg', 'mig_same', 'mig_prev_sunbelt',
                    'num_emp', 'fam_under_18', 'country_father', 'country_mother', 'country_self', 'citizenship',
                    'own_or_self', 'vet_question', 'vet_benefits', 'weeks_worked', 'year', 'income_50k']

    train_df = pd.read_csv('./DATA/census-income.data', delimiter=',', header=None, index_col=None, names=column_names)
    test_df = pd.read_csv('./DATA/census-income.test', delimiter=',', header=None, index_col=None, names=column_names)

    label_columns = ['income_50k', 'marital_stat']
    categorical_columns = ['class_worker', 'det_ind_code', 'det_occ_code', 'education', 'hs_college', 'major_ind_code',
                           'major_occ_code', 'race', 'hisp_origin', 'sex', 'union_member', 'unemp_reason',
                           'full_or_part_emp', 'tax_filer_stat', 'region_prev_res', 'state_prev_res', 'det_hh_fam_stat',
                           'det_hh_summ', 'mig_chg_msa', 'mig_chg_reg', 'mig_move_reg', 'mig_same', 'mig_prev_sunbelt',
                           'fam_under_18', 'country_father', 'country_mother', 'country_self', 'citizenship',
                           'vet_question']
    train_transformed = pd.get_dummies(train_df.drop(label_columns, axis=1), columns=categorical_columns)
    test_transformed = pd.get_dummies(test_df.drop(label_columns, axis=1), columns=categorical_columns)
    train_labels = train_df[label_columns]
    test_labels = test_df[label_columns]

    test_transformed['det_hh_fam_stat_ Grandchild <18 ever marr not in subfamily'] = 0

    train_income = to_categorical((train_labels.income_50k == ' 50000+.').astype(int), num_classes=2)
    train_marital = to_categorical((train_labels.marital_stat == ' Never married').astype(int), num_classes=2)
    other_income = to_categorical((test_labels.income_50k == ' 50000+.').astype(int), num_classes=2)
    other_marital = to_categorical((test_labels.marital_stat == ' Never married').astype(int), num_classes=2)

    dict_outputs = {'income': train_income.shape[1], 'marital': train_marital.shape[1]}
    dict_train_labels = {'income': train_income, 'marital': train_marital}
    dict_other_labels = {'income': other_income, 'marital': other_marital}

    validation_indices = test_transformed.sample(frac=0.5, replace=False, random_state=seed).index
    test_indices = list(set(test_transformed.index) - set(validation_indices))
    validation_data = test_transformed.iloc[validation_indices]
    validation_label = [dict_other_labels[key][validation_indices] for key in sorted(dict_other_labels.keys())]
    test_data = test_transformed.iloc[test_indices]
    test_label = [dict_other_labels[key][test_indices] for key in sorted(dict_other_labels.keys())]
    train_data = train_transformed
    train_label = [dict_train_labels[key] for key in sorted(dict_train_labels.keys())]

    return train_data, train_label, validation_data, validation_label, test_data, test_label, dict_outputs

def build_mmoe(input_dim, num_experts=6, experts_out=16, experts_hidden=32, towers_hidden=8, num_tasks=2):
    inputs = Input(shape=(input_dim,))

    # Experts
    experts = []
    for i in range(num_experts):
        x = Dense(experts_hidden, activation='relu')(inputs)
        x = Dropout(0.3)(x)
        x = Dense(experts_out)(x)
        experts.append(x)

    # 堆叠 expert 输出: (batch, num_experts, experts_out)
    expert_stack = Lambda(lambda x: K.stack(x, axis=1))(experts)

    outputs = []
    for t in range(num_tasks):
        # gate (batch, num_experts)
        gate = Dense(num_experts, activation='softmax')(inputs)
        gate = Lambda(lambda g: K.expand_dims(g, axis=-1))(gate)  # (batch, num_experts, 1)

        # 加权求和 experts
        tower_input = Lambda(lambda z: K.sum(z[0] * z[1], axis=1))([expert_stack, gate])

        # Tower
        tower = Dense(towers_hidden, activation='relu')(tower_input)
        tower = Dropout(0.4)(tower)
        tower_output = Dense(1, activation='sigmoid', name=f'task{t}')(tower)
        outputs.append(tower_output)

    model = Model(inputs=inputs, outputs=outputs)
    return model


# 数据准备
train_data, train_label, val_data, val_label, test_data, test_label, output_info = data_preparation()
input_dim = train_data.shape[1]

# 建模
model = build_mmoe(input_dim = input_dim, num_experts = 8, experts_out = 16,
                   experts_hidden = 16, towers_hidden = 8, num_tasks = 2)

# 编译模型
model.compile(
    optimizer=tf.train.AdamOptimizer(1e-4),
    loss='binary_crossentropy',
    metrics=[auc_roc]
)

print(model.summary())




# 训练
history = model.fit(train_data.values,
                    {'task0': train_label[0][:,1], 'task1': train_label[1][:,1]},
                    validation_data=(val_data.values, {'task0': val_label[0][:,1], 'task1': val_label[1][:,1]}),
                    epochs = 50,
                    batch_size = batch_size)

# 测试
res = model.evaluate(test_data.values, {'task0': test_label[0][:,1], 'task1': test_label[1][:,1]})
print("Test results:", res)

Using TensorFlow backend.







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
__________________________________________________________________________________________________
Layer (type



Train on 199523 samples, validate on 49881 samples
Epoch 1/50




2025-08-20 09:15:49.969689: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2025-08-20 09:15:49.987001: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2687995000 Hz
2025-08-20 09:15:49.994061: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x35e93a40 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2025-08-20 09:15:49.994108: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2025-08-20 09:15:50.006766: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1


2025-08-20 09:15:50.990354: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-08-20 09:15:50.990674: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x32023a80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-08-20 09:15:50.990706: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2025-08-20 09:15:50.997492: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-08-20 09:15:50.997584: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3060 Laptop GPU major: 8 minor: 6 memoryClockRate(GH


199523/199523 [==============================] - 7s 33us/step - loss: 6.4799 - task0_loss: 1.1427 - task1_loss: 5.3373 - task0_auc_roc: 0.5145 - task1_auc_roc: 0.5855 - val_loss: 2.6807 - val_task0_loss: 0.9949 - val_task1_loss: 1.6858 - val_task0_auc_roc: 0.4982 - val_task1_auc_roc: 0.8445
Epoch 2/50
199523/199523 [==============================] - 5s 23us/step - loss: 3.6055 - task0_loss: 1.0512 - task1_loss: 2.5543 - task0_auc_roc: 0.5096 - task1_auc_roc: 0.7631 - val_loss: 2.3903 - val_task0_loss: 0.9946 - val_task1_loss: 1.3957 - val_task0_auc_roc: 0.4969 - val_task1_auc_roc: 0.8566
Epoch 3/50
199523/199523 [==============================] - 5s 24us/step - loss: 2.6937 - task0_loss: 0.9866 - task1_loss: 1.7071 - task0_auc_roc: 0.5093 - task1_auc_roc: 0.8035 - val_loss: 2.3015 - val_task0_loss: 0.9936 - val_task1_loss: 1.3079 - val_task0_auc_roc: 0.5000 - val_task1_auc_roc: 0.8152
Epoch 4/50
199523/199523 [==============================] - 5s 23us/step - loss: 2.2530 - task0_loss:

199523/199523 [==============================] - 5s 23us/step - loss: 0.5483 - task0_loss: 0.2267 - task1_loss: 0.3216 - task0_auc_roc: 0.8021 - task1_auc_roc: 0.9341 - val_loss: 0.4428 - val_task0_loss: 0.1723 - val_task1_loss: 0.2705 - val_task0_auc_roc: 0.8831 - val_task1_auc_roc: 0.9622
Epoch 28/50
199523/199523 [==============================] - 5s 24us/step - loss: 0.5354 - task0_loss: 0.2228 - task1_loss: 0.3127 - task0_auc_roc: 0.8046 - task1_auc_roc: 0.9373 - val_loss: 0.4446 - val_task0_loss: 0.1726 - val_task1_loss: 0.2721 - val_task0_auc_roc: 0.8812 - val_task1_auc_roc: 0.9633
Epoch 29/50
199523/199523 [==============================] - 5s 23us/step - loss: 0.5221 - task0_loss: 0.2177 - task1_loss: 0.3044 - task0_auc_roc: 0.8106 - task1_auc_roc: 0.9404 - val_loss: 0.4359 - val_task0_loss: 0.1714 - val_task1_loss: 0.2645 - val_task0_auc_roc: 0.8852 - val_task1_auc_roc: 0.9646
Epoch 30/50
199523/199523 [==============================] - 4s 22us/step - loss: 0.5131 - task0_los